# SageMaker Inference: BiEncoder RoBerta

[KLUE RoBERTa](https://huggingface.co/klue/roberta-base) 모델을 SageMaker Endpoint로 배포하고 추론합니다.

---

## [선수 작업] AWS Role 정보를 .env 파일에 아래와 같이 저장
```
SAGEMAKER_ROLE_ARN=arn:aws:iam::XXXXXX:role/gonsoomoon-sm-inference
```

## 0 환경 확인

In [ ]:
! which python

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')


In [ ]:

from dotenv import load_dotenv
import os

load_dotenv('../.env')
SAGEMAKER_ROLE_ARN = os.getenv('SAGEMAKER_ROLE_ARN')

## 1. 환경 설정

In [ ]:
import json
import time
import boto3
import sagemaker
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sagemaker_session = sagemaker.Session()
role = SAGEMAKER_ROLE_ARN
bucket = sagemaker_session.default_bucket()

print(f"Bucket: {bucket}")

## 2. 모델 아티팩트 생성 및 S3 업로드

model.tar.gz 구조로 생성을 하면 , SageMaker 가 이를 인지 합니다.
model.tar.gz 구조:
```
model.tar.gz/
├── config.json
├── model.safetensors
├── special_tokens_map.json
├── tokenizer.json
├── tokenizer_config.json
├── vocab.txt
└── code/
    ├── inference.py
    └── requirements.txt
```

참조: [SageMaker PyTorch Documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#deploy-pytorch-models)

### model.tar.gz 파일 생성

In [ ]:
!rm -rf ../model_artifact
!mkdir -p ../model_artifact/code
!cp ../src/inference.py ../model_artifact/code/
!cp ../src/requirements.txt ../model_artifact/code/
!cp ../model/* ../model_artifact/

!cd ../model_artifact && tar -czf ../model.tar.gz *

model_artifact_s3_uri = f's3://{bucket}/klue-roberta-inference/model/model.tar.gz'
!aws s3 cp ../model.tar.gz {model_artifact_s3_uri}

print(f"Model uploaded to: {model_artifact_s3_uri}")

## 3. SageMaker Endpoint 생성

### SageMaker Endpoint 생성

In [ ]:
endpoint_name = "endpoint-dual-encoder-{}".format(int(time.time()))

pytorch_model = PyTorchModel(model_data=model_artifact_s3_uri,
                                   role=role,
                                   entry_point='inference.py',
                                   source_dir = '../src',
                                   framework_version='2.5',
                                   py_version='py311',
                                   model_server_workers=1,
                                  )

predictor = pytorch_model.deploy(
                           instance_type="ml.g4dn.xlarge", 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True,
                           log = False,
                        )

## 4. Endpoint 추론

### 싱글 샘플 추론

In [ ]:
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

# BiEncoder: 단일 쿼리-문서 쌍 테스트
result = predictor.predict({
    "queries": ["맛있는 한국 전통 음식 김치찌개"],
    "documents": ["김치찌개와 된장찌개는 한국의 대표 전통 음식입니다."]
})

print(f"Query embeddings shape: ({result['num_queries']}, {result['embedding_dim']})")
print(f"Document embeddings shape: ({result['num_documents']}, {result['embedding_dim']})")


import numpy as np

# 유사도 계산 (이미 정규화되어 있으므로 내적만 계산)
query_emb = np.array(result["query_embeddings"])[0]
doc_emb = np.array(result["doc_embeddings"])[0]

similarity = np.dot(query_emb, doc_emb)

print(f"\nCosine similarity: {similarity:.4f}")

### 3개의 샘플 추론 및 유사도 비교

In [ ]:
import sys
from src.utils import test_biencoder_pairs

query_doc_pairs = [
    (
        "맛있는 한국 전통 음식 김치찌개",
        "김치찌개와 된장찌개는 한국의 대표 전통 음식입니다."
    ),
    (
        "최신 기술 발전",
        "인공지능 기술이 빠르게 발전하고 있습니다."
    ),
    (
        "색깔",
        "파리의 에펠탑은 프랑스의 상징입니다."
    )
]

# BiEncoder 쌍별 유사도 테스트 실행
test_biencoder_pairs(predictor, query_doc_pairs)

### 8개 쿼리-문서 쌍 배치 추론

In [ ]:
batch_result = predictor.predict({
    "queries": [
        "맛있는 한국 전통 음식 김치찌개",
        "최신 기술 발전", 
        "색깔",
        "여행 계획",
        "스포츠 경기",
        "영화 추천",
        "날씨 정보",
        "건강 관리"
    ],
    "documents": [
        "김치찌개와 된장찌개는 한국의 대표 전통 음식입니다.",
        "인공지능 기술이 빠르게 발전하고 있습니다.",
        "파리의 에펠탑은 프랑스의 상징입니다.",
        "제주도는 한국의 인기 여행지입니다.",
        "축구 경기가 오늘 저녁에 있습니다.",
        "최근 개봉한 영화가 좋은 평가를 받고 있습니다.",
        "내일은 맑은 날씨가 예상됩니다.",
        "규칙적인 운동이 건강에 좋습니다."
    ]
})

print(f"Batch inference completed:")
print(f"  Queries: {batch_result['num_queries']}")
print(f"  Documents: {batch_result['num_documents']}")
print(f"  Embedding dim: {batch_result['embedding_dim']}\n")

# 각 쌍의 코사인 유사도 계산
query_embs = np.array(batch_result['query_embeddings'])
doc_embs = np.array(batch_result['doc_embeddings'])

print("Pair-wise cosine similarities:")
for i in range(len(query_embs)):
    similarity = np.dot(query_embs[i], doc_embs[i])
    print(f"  Pair {i+1}: {similarity:.4f}")

## 5. Boto3 invoke_endpoint() 로 추론

In [ ]:
import boto3
import json
import time
import numpy as np

runtime_client = boto3.Session().client('sagemaker-runtime')

def invoke_endpoint(runtime_client, endpoint_name, payload, content_type):
    '''
    SageMaker 엔드포인트 호출
    '''
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType=content_type, 
        Body=payload,
    )
    
    result = response['Body'].read().decode()
    return json.loads(result)

# BiEncoder 페이로드 생성
payload = {
    "queries": ["맛있는 한국 전통 음식 김치찌개"],
    "documents": ["김치찌개와 된장찌개는 한국의 대표 전통 음식입니다."]
}

payload_dump = json.dumps(payload)

# 엔드포인트 호출
start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print(f"Query embeddings shape: ({result['num_queries']}, {result['embedding_dim']})")
print(f"Document embeddings shape: ({result['num_documents']}, {result['embedding_dim']})")

# 유사도 계산
query_emb = np.array(result["query_embeddings"])[0]
doc_emb = np.array(result["doc_embeddings"])[0]
similarity = np.dot(query_emb, doc_emb)

print(f"Cosine similarity: {similarity:.4f}")

## 6. 엔드 포인트 제거


In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)
print("✅ Endpoint 삭제 완료")